In [4]:
import torch

class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
        self.biases = torch.zeros((1, n_neurons))
        self.output = None

    def forward(self, inputs):
        self.output = torch.matmul(inputs, self.weights) + self.biases
        return self.output

class ActivationReLU:
    def forward(self, inputs):
        self.output = torch.relu(inputs)
        return self.output

class ActivationSigmoid:
    def forward(self, inputs):
        self.output = torch.sigmoid(inputs)
        return self.output
class ActivationSoftmax:
    def forward(self, inputs):
        self.output = torch.nn.functional.softmax(inputs, dim=1)
        return self.output

# Input data
X = torch.tensor([0.4, 0.9])

# Create layers and activations
hidden_layer_1 = DenseLayer(2, 2)
activation1 = ActivationReLU()
output_layer = DenseLayer(2, 2)
activation2 = ActivationSigmoid()

# Forward pass
hidden_layer_1_output = hidden_layer_1.forward(X)
activation1_output = activation1.forward(hidden_layer_1_output)
output_layer_output = output_layer.forward(activation1_output)
activation2_output = activation2.forward(output_layer_output)

print("Activation2 Output:", activation2_output)


Activation2 Output: tensor([[0.5000, 0.5000]])


In [7]:
def accuracy(predictions, targets):
    if predictions.shape[-1] > 1:
        _, predicted_classes = torch.max(predictions, 1)
    else:
        predicted_classes = torch.round(predictions)

    correct_predictions = (predicted_classes == targets).sum().item()
    total_samples = len(targets)

    accuracy_value = (correct_predictions / total_samples) * 100.0

    return accuracy_value
def mean_squared_error(predictions, targets):
    return torch.mean((predictions - targets)**2)

In [11]:
def back_propagation(X, y, lr=0.01):
    # Forward pass
    hidden_layer_1_output = hidden_layer_1.forward(X)
    activation1_output = activation1.forward(hidden_layer_1_output)
    hidden_layer_2_output = hidden_layer_2.forward(activation1_output)
    activation2_output = activation2.forward(hidden_layer_2_output)
    output_layer_output = output_layer.forward(activation2_output)

    # Backward pass
    back_output = output_layer_output - y
    back_hidden_2 = torch.matmul(back_output, output_layer.weights.t()) * (hidden_layer_2_output > 0).float()
    back_hidden_1 = torch.matmul(back_hidden_2, hidden_layer_2.weights.t()) * (hidden_layer_1_output > 0).float()

    # Update weights and biases
    output_layer.weights -= lr * torch.matmul(activation2_output.t(), back_output)
    output_layer.biases -= lr * back_output.sum(dim=0, keepdim=True)

    hidden_layer_2.weights -= lr * torch.matmul(activation1_output.t(), back_hidden_2)
    hidden_layer_2.biases -= lr * back_hidden_2.sum(dim=0, keepdim=True)

    hidden_layer_1.weights -= lr * torch.matmul(X.reshape(1, -1).t(), back_hidden_1)
    hidden_layer_1.biases -= lr * back_hidden_1.sum(dim=0, keepdim=True)

# Input data
X = torch.tensor([0.4, 0.9]).reshape(1, -1)
y = torch.tensor([1, 0]).reshape(1, -1)

# Create layers and activations
hidden_layer_1 = DenseLayer(2, 2)
activation1 = ActivationReLU()
hidden_layer_2 = DenseLayer(2, 2)
activation2 = ActivationReLU()
output_layer = DenseLayer(2, 2)
softmax_activation = ActivationSoftmax()

# Training loop
for epoch in range(1000):
    # Forward and backward pass
    back_propagation(X, y)

    if epoch % 100 == 0:
        # Print loss and accuracy every 100 epochs
        hidden_layer_1_output = hidden_layer_1.forward(X)
        activation1_output = activation1.forward(hidden_layer_1_output)
        hidden_layer_2_output = hidden_layer_2.forward(activation1_output)
        activation2_output = activation2.forward(hidden_layer_2_output)
        output_layer_output = output_layer.forward(activation2_output)

        loss = mean_squared_error(output_layer_output, y)
        acc = accuracy(output_layer_output, torch.round(y))
        print(f"Epoch {epoch}, Loss: {loss.item()}, Accuracy: {acc}%")

# Test the final output
hidden_layer_1_output = hidden_layer_1.forward(X)
activation1_output = activation1.forward(hidden_layer_1_output)
hidden_layer_2_output = hidden_layer_2.forward(activation1_output)
activation2_output = activation2.forward(hidden_layer_2_output)
final_output = output_layer.forward(activation2_output)
final_softmax = softmax_activation.forward(final_output)

print("Final Output:", final_output)
print("Final Softmax Output:", final_softmax)

Epoch 0, Loss: 0.4900476634502411, Accuracy: 100.0%
Epoch 100, Loss: 0.0656316950917244, Accuracy: 100.0%
Epoch 200, Loss: 0.0087873674929142, Accuracy: 100.0%
Epoch 300, Loss: 0.0011762842768803239, Accuracy: 100.0%
Epoch 400, Loss: 0.0001574439520481974, Accuracy: 100.0%
Epoch 500, Loss: 2.1073941752547398e-05, Accuracy: 100.0%
Epoch 600, Loss: 2.820472673192853e-06, Accuracy: 100.0%
Epoch 700, Loss: 3.776120820475626e-07, Accuracy: 100.0%
Epoch 800, Loss: 5.048337570201511e-08, Accuracy: 100.0%
Epoch 900, Loss: 6.7476868537141854e-09, Accuracy: 100.0%
Final Output: tensor([[9.9996e-01, 7.4346e-08]])
Final Softmax Output: tensor([[0.7311, 0.2689]])
